In [28]:
# https://github.com/agupta98/Tensorflow
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
# read the data set and display the first 5 records
census = pd.read_csv("census_data.csv")
census.head(5)

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [3]:
# print the unique values
census["income_bracket"].unique()

array([' <=50K', ' >50K'], dtype=object)

In [5]:
# changing label values for income_bracket
def label_fix(label):
    if label == ' <=50K':
        return 0
    else:
        return 1
    
census["income_bracket"] = census["income_bracket"].apply(label_fix)
census["income_bracket"]

0        0
1        0
2        0
3        0
4        0
        ..
32556    0
32557    1
32558    0
32559    0
32560    1
Name: income_bracket, Length: 32561, dtype: int64

In [8]:
# split the data for training and testing
x_data = census.drop("income_bracket", axis=1)
y_label = census["income_bracket"]
x_train, x_test, y_train, y_test = train_test_split(x_data, y_label, test_size=0.3, random_state=101)

In [9]:
# encoding categorical columns

gender = tf.feature_column.categorical_column_with_vocabulary_list("gender", ["Female", "Male"])
occupation = tf.feature_column.categorical_column_with_hash_bucket("occupation", hash_bucket_size=1000)
marital_status = tf.feature_column.categorical_column_with_hash_bucket("marital_status", hash_bucket_size=1000)
relationship = tf.feature_column.categorical_column_with_hash_bucket("relationship", hash_bucket_size=1000)
education = tf.feature_column.categorical_column_with_hash_bucket("education", hash_bucket_size=1000)
workclass = tf.feature_column.categorical_column_with_hash_bucket("workclass", hash_bucket_size=1000)
native_country = tf.feature_column.categorical_column_with_hash_bucket("native_country", hash_bucket_size=1000)

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.
Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


In [13]:
# preprocessing numeric continuous values

age = tf.feature_column.numeric_column("age")
education_num = tf.feature_column.numeric_column("education_num")
capital_gain = tf.feature_column.numeric_column("capital_gain")
capital_loss = tf.feature_column.numeric_column("capital_loss")
hours_per_week = tf.feature_column.numeric_column("hours_per_week")

In [14]:
# put all features in a list | creating a vector with features

feature_cols = [gender, occupation, marital_status, relationship, education, workclass, native_country, 
                age, education_num, capital_gain, capital_loss, hours_per_week]

In [20]:
# create the input function

input_function = tf.compat.v1.estimator.inputs.pandas_input_fn(x=x_train, y=y_train, batch_size=100, 
                                                               num_epochs=None, shuffle=True)

In [22]:
# building the model

model = tf.estimator.LinearClassifier(feature_columns=feature_cols)
model.train(input_fn=input_function, steps=5000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\SUJITH~1\\AppData\\Local\\Temp\\tmpdkc0j2p7', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Use tf.keras ins

In [23]:
# create predict input function
predict_function = tf.compat.v1.estimator.inputs.pandas_input_fn(x=x_test, batch_size=len(x_test), shuffle=False)

In [25]:
# make the final prediction
predictions = list(model.predict(input_fn=predict_function))
predictions[0]

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\SUJITH~1\AppData\Local\Temp\tmpdkc0j2p7\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


{'logits': array([-1.1773127], dtype=float32),
 'logistic': array([0.23553571], dtype=float32),
 'probabilities': array([0.76446426, 0.23553571], dtype=float32),
 'class_ids': array([0], dtype=int64),
 'classes': array([b'0'], dtype=object),
 'all_class_ids': array([0, 1]),
 'all_classes': array([b'0', b'1'], dtype=object)}

In [27]:
# formatting the prediction output and print first 10
final_pred = []
for pred in predictions:
    final_pred.append(pred["class_ids"][0])
    
final_pred[:10]

[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]

In [31]:
# print the accuracy report
print(classification_report(y_test, final_pred))

              precision    recall  f1-score   support

           0       0.88      0.90      0.89      7436
           1       0.67      0.61      0.64      2333

    accuracy                           0.83      9769
   macro avg       0.77      0.76      0.76      9769
weighted avg       0.83      0.83      0.83      9769

